In [1]:
import pandas as pd
from pathlib import Path

In [2]:
ls

 Volume in drive C is OS
 Volume Serial Number is 5E7E-789E

 Directory of C:\Users\Gary2\Partners HealthCare Dropbox\Gurdip Uppal\research_bwh\MCSPACE_FINAL\MCSPACE\mcspace\data\mouse_experiments

08/26/2024  09:20 AM    <DIR>          .
08/20/2024  03:34 PM    <DIR>          ..
08/26/2024  09:18 AM    <DIR>          .ipynb_checkpoints
08/20/2024  03:34 PM    <DIR>          _raw_data
06/11/2024  10:10 AM           109,967 combined_zotus .fa
08/13/2024  01:22 PM        12,618,963 mouse_counts.csv.gz
08/20/2024  03:34 PM    <DIR>          output
06/11/2024  12:13 PM           157,797 tax.csv
08/20/2024  03:34 PM    <DIR>          tmp
08/26/2024  09:20 AM               805 update_taxonomy.ipynb
06/11/2024  10:10 AM           109,967 zotus.fa
               5 File(s)     12,997,499 bytes
               6 Dir(s)  990,304,215,040 bytes free


In [3]:
pwd

'C:\\Users\\Gary2\\Partners HealthCare Dropbox\\Gurdip Uppal\\research_bwh\\MCSPACE_FINAL\\MCSPACE\\mcspace\\data\\mouse_experiments'

In [5]:
basepath = Path("./")
outpath = basepath / "taxonomy"
outpath.mkdir(exist_ok=True, parents=True)

In [6]:
tax = pd.read_csv("tax.csv")

In [7]:
tax.head()

,Otu,domain,dconf,phylum,pconf,class,cconf,order,oconf,family,fconf,genus,gconf
0,Otu1,Bacteria,1.0,Firmicutes,1.0,Bacilli,1.0,Lactobacillales,1.0,Lactobacillaceae,1.00,Lactobacillus,0.99
1,Otu7,Bacteria,1.0,Firmicutes,1.0,Clostridia,1.0,Clostridiales,1.0,Lachnospiraceae,1.00,Clostridium_XlVa,0.68
2,Otu11,Bacteria,1.0,Actinobacteria,1.0,Coriobacteriia,1.0,Eggerthellales,1.0,Eggerthellaceae,1.00,Adlercreutzia,0.85
3,Otu8,Bacteria,1.0,Bacteroidetes,1.0,Bacteroidia,1.0,Bacteroidales,1.0,Muribaculaceae,0.99,Duncaniella,0.99
4,Otu2,Bacteria,1.0,Verrucomicrobia,1.0,Verrucomicrobiae,1.0,Verrucomicrobiales,1.0,Akkermansiaceae,1.00,Akkermansia,1.00


In [8]:
def apply_taxonomy_threshold(taxonomy, threshold=0.5):
    ranks = ['domain', 'phylum', 'class', 'order', 'family', 'genus']
    conf = ['dconf', 'pconf', 'cconf', 'oconf', 'fconf', 'gconf']
    
    taxcopy = taxonomy.reset_index()
    ntaxa = taxcopy.shape[0]
    for i in range(ntaxa):
        for r,c in zip(ranks, conf):
            if taxcopy.loc[i,c] < threshold:
                taxcopy.loc[i,r] = 'na'
    
    ptaxa = taxcopy.set_index("Otu")
    ptaxa2 = ptaxa[ranks]
    
    mapper = {x:x.capitalize() for x in list(ptaxa2.columns)}
    ptaxa3 = ptaxa2.rename(columns=mapper)
    return ptaxa3

In [9]:
filttax = apply_taxonomy_threshold(tax)

In [10]:
filttax.head()

,Domain,Phylum,Class,Order,Family,Genus
Otu,,,,,,
Otu1,Bacteria,Firmicutes,Bacilli,Lactobacillales,Lactobacillaceae,Lactobacillus
Otu7,Bacteria,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,Clostridium_XlVa
Otu11,Bacteria,Actinobacteria,Coriobacteriia,Eggerthellales,Eggerthellaceae,Adlercreutzia
Otu8,Bacteria,Bacteroidetes,Bacteroidia,Bacteroidales,Muribaculaceae,Duncaniella
Otu2,Bacteria,Verrucomicrobia,Verrucomicrobiae,Verrucomicrobiales,Akkermansiaceae,Akkermansia


#### update names

In [13]:
taxonomy = filttax.copy()

In [14]:
name_updates = {'Otu10': {'Species': 'Faecalibaculum rodentium'},
'Otu17': {'Genus': 'Roseburia'},
'Otu6': {'Species': 'Eubacterium coprostanoligenes'},
'Otu20': {'Species': 'Muribaculum gordoncarteri'},
'Otu15': {'Genus': 'Eisenbergiella'},
'Otu43': {'Family': 'Lachnospiraceae'}}

In [15]:
taxonomy['Species'] = 'na'

In [16]:
for oidx in name_updates.keys():
    replace = name_updates[oidx]
    key = list(replace.keys())[0]
    newname = replace[key]
    taxonomy.loc[oidx,key] = newname

In [17]:
taxonomy.loc['Otu10',:]

Domain                     Bacteria
Phylum                   Firmicutes
Class              Erysipelotrichia
Order            Erysipelotrichales
Family          Erysipelotrichaceae
Genus                Faecalibaculum
Species    Faecalibaculum rodentium
Name: Otu10, dtype: object

In [18]:
taxonomy.loc['Otu6',:]

Domain                          Bacteria
Phylum                        Firmicutes
Class                         Clostridia
Order                      Clostridiales
Family                    Eubacteriaceae
Genus                        Eubacterium
Species    Eubacterium coprostanoligenes
Name: Otu6, dtype: object

In [19]:
taxonomy['Family'].unique()

array(['Lactobacillaceae', 'Lachnospiraceae', 'Eggerthellaceae',
       'Muribaculaceae', 'Akkermansiaceae', 'Eubacteriaceae',
       'Ruminococcaceae', 'Erysipelotrichaceae', 'Streptococcaceae',
       'Bacillaceae_1', 'na', 'Peptostreptococcaceae',
       'Porphyromonadaceae', 'Clostridiaceae_1', 'Acholeplasmataceae',
       'Dysgonomonadaceae', 'Flavobacteriaceae', 'Poaceae',
       'Clostridiales_Incertae_Sedis_XIII', 'Prevotellaceae',
       'Chloroplast', 'Tissierellaceae', 'Clostridiales_incertae_sedis',
       'Bacteroidaceae', 'Rikenellaceae', 'Bifidobacteriaceae',
       'Deferribacteraceae', 'Sphingobacteriaceae', 'Sphingomonadaceae',
       'Clostridiaceae_3', 'Marinilabiliaceae', 'Odoribacteraceae',
       'Bradyrhizobiaceae', 'Staphylococcaceae', 'Microbacteriaceae',
       'Coriobacteriaceae', 'Phyllobacteriaceae', 'Chlorobiaceae',
       'Atopobiaceae', 'Desulfovibrionaceae', 'Synergistaceae',
       'Spirochaetaceae', 'Fulvivirgaceae', 'Cytophagaceae',
       'Enteroba

In [20]:
taxonomy.to_csv(outpath / "taxonomy.csv")

In [21]:
ls 

 Volume in drive C is OS
 Volume Serial Number is 5E7E-789E

 Directory of C:\Users\Gary2\Partners HealthCare Dropbox\Gurdip Uppal\research_bwh\MCSPACE_FINAL\MCSPACE\mcspace\data\mouse_experiments

08/26/2024  09:26 AM    <DIR>          .
08/20/2024  03:34 PM    <DIR>          ..
08/26/2024  09:18 AM    <DIR>          .ipynb_checkpoints
08/20/2024  03:34 PM    <DIR>          _raw_data
06/11/2024  10:10 AM           109,967 combined_zotus .fa
08/13/2024  01:22 PM        12,618,963 mouse_counts.csv.gz
08/20/2024  03:34 PM    <DIR>          output
06/11/2024  12:13 PM           157,797 tax.csv
08/26/2024  09:27 AM    <DIR>          taxonomy
08/20/2024  03:34 PM    <DIR>          tmp
08/26/2024  09:26 AM            16,058 update_taxonomy.ipynb
06/11/2024  10:10 AM           109,967 zotus.fa
               5 File(s)     13,012,752 bytes
               7 Dir(s)  990,319,476,736 bytes free


In [22]:
ls "taxonomy/"

 Volume in drive C is OS
 Volume Serial Number is 5E7E-789E

 Directory of C:\Users\Gary2\Partners HealthCare Dropbox\Gurdip Uppal\research_bwh\MCSPACE_FINAL\MCSPACE\mcspace\data\mouse_experiments\taxonomy

08/26/2024  09:27 AM    <DIR>          .
08/26/2024  09:26 AM    <DIR>          ..
08/26/2024  09:27 AM            93,514 taxonomy.csv
               1 File(s)         93,514 bytes
               2 Dir(s)  990,319,280,128 bytes free
